In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *

In [ ]:
df = pd.read_csv(cache_dir+'pairs_v2.tsv', sep='\t')

In [ ]:
num = df[(df['related']==True) & (df['vec_sim']==0)].shape[0]

In [ ]:
df = df[~((df['related']==True) & (df['vec_sim']==0))]

In [ ]:
df[df['vec_sim'].isnull()]

In [ ]:
df = df[~df.isin(df[df['related']==False].sample(num))].dropna()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import KFold
X = df[['vec_sim', 'jac_sim', 'len_sim', 'top_sim']].values
y = df[['related']].values.ravel()


fold = 10
kf = KFold(n_splits=fold, random_state=42, shuffle=True)
score = 0.0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svm = SVC()
    svm.fit(X_train, y_train)
    score += svm.score(X_test, y_test)
print(score/fold)